In [2]:
from wptherml.wpml import multilayer
from wptherml.datalib import datalib
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import basinhopping
import numpy as np
from scipy.interpolate import UnivariateSpline

### Re-define structure:
### layer 1 can be AlN, HfO2, TiO2, SiO2
### layer 2 can be AlN, HfO2, TiO2, SiO2... just not the same as AlN
### layer 3 can be W, TiN, Pd, Pt
structure = {
        ### No material actually called "Lorentz", so just use SiO2 as a placeholder
        ### that we can change later

 'Material_List' : ['Air', 'AlN','SiO2', 'Si','Air'],
        ### Thicknesses just chosen arbitrarily, replace with "optimal" values
        'Thickness_List': [0, 70e-9, 70e-9, 2000e-9, 0],
        ### add a number to Gradient_List to optimize over more layers
        'Gradient_List': [1,2],
        'Lambda_List': [300e-9, 4000e-9, 1000],
        }






cc = multilayer(structure)
length = len(cc.conversion_efficiency_grad)

### a function to take an array of values x_1, ..., x_8 as defined above,
### re-define the multi-layer structure according to its values, and
### then update the luminous efficiency accordingly:
def update_multilayer(x):
    ### recall x_1 is omega_p, x_2 is omega_0, x_3 is gamma
    #cc.layer_lorentz(1, omega_p, omega_0, gamma)
    for i in range(0,len(x)):
        cc.d[i+1] = x[i]*1e-9
    #cc.d[3] = x[2]*1e-9
    #print("Thickness is ",cc.d[1])
    ### now we have the new structure, update fresnel quantities
    cc.fresnel()
    ### now we have new emissivity, update thermal emission
    cc.pv_conversion_efficiency()

    ### return luminous efficiency
    return -cc.conversion_efficiency_val*100

def analytic_grad(x0):
    cur = update_multilayer(x0)
    cc.fresnel_prime()
    cc.pv_conversion_efficiency_prime()
    g = cc.conversion_efficiency_grad
    return -g*1e-9

def BuildGradient(x0):
    #h0 = 0.0001*x0
    dim = len(x0)
    h0 = 0.1e-9*np.ones(dim)
    g = np.zeros(dim)
    for i in range(0,dim):
        xpass = np.copy(x0)
        fx = x0[i] + h0[i]
        bx = x0[i] - h0[i]
        xpass[i] = fx
        efx = update_multilayer(xpass)
        xpass[i] = bx
        ebx = update_multilayer(xpass)

        run = 2*h0[i]
        g[i] = (efx-ebx)/run
    return g



def SuperFunc(x):
    en = update_multilayer(x)
    gr = analytic_grad(x)
    return en, gr


import time
def print_fun(x, f, accepted):
    c_time = time.time()
    ### note: print x[0]... x[L]
    print(f,",",c_time)

def my_take_step(x):
    xnew = np.copy(x)
    dim = len(xnew)
    for i in range(0,dim):
        rn = 50.*np.abs(np.random.randn())
        xnew[i] = rn
    return xnew


class MyBounds(object):
      ### note xmax and xmin need to have as many elements as there are thicknesses that are varied
    def __init__(self, xmax=49.01*np.ones(length), xmin=0.01*np.ones(length)):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

# the bounds for L-BFGS-B updates!
bfgs_xmin = 0.01*np.ones(length)
bfgs_xmax = 49.01*np.ones(length)

# rewrite the bounds in the way required by L-BFGS-B
bfgs_bounds = [(low, high) for low, high in zip(bfgs_xmin, bfgs_xmax)]

minimizer_kwargs = {"method": "L-BFGS-B", "jac": True, "bounds": bfgs_bounds}
mybounds = MyBounds()

xs = np.ones(length)*20.

ret = basinhopping(SuperFunc, xs, minimizer_kwargs=minimizer_kwargs, niter=100, take_step=my_take_step, callback=print_fun, accept_test=mybounds)

print(ret.x)
print(update_multilayer(ret.x))



 Temperature not specified!
 Proceeding with default T = 300 K


IndexError: too many indices for array